# 📘 2권 3장: 웹 애플리케이션 개발 (Flask & Django)

## 📌 목차
13.1 웹 개발 개념 및 기본 원리  
13.2 Flask를 사용한 웹 애플리케이션 개발  
13.3 Django를 사용한 대형 웹 프로젝트 구축  
13.4 REST API 개발 및 테스트  
13.5 OAuth 및 사용자 인증 처리  

## 13.1 웹 개발의 핵심 구성 요소

### ✅ 13.1.1 웹 애플리케이션 아키텍처
1. **프론트엔드**
   - HTML: 웹 페이지 구조
   - CSS: 스타일링
   - JavaScript: 동적 기능

2. **백엔드**
   - 서버 로직
   - 데이터 처리
   - API 구현

3. **데이터베이스**
   - 데이터 저장
   - 쿼리 처리
   - 데이터 관리

In [ ]:
# Flask 기본 설정
from flask import Flask, render_template, jsonify
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///app.db'
db = SQLAlchemy(app)

class User(db.Model):
    """사용자 모델"""
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)

@app.route('/')
def home():
    """홈페이지 라우트"""
    return render_template('index.html')

@app.route('/api/users')
def get_users():
    """사용자 목록 API"""
    users = User.query.all()
    return jsonify([{'username': user.username, 'email': user.email} for user in users])

## 13.2 Flask 웹 애플리케이션 개발

### ✅ 13.2.1 Flask 프로젝트 구조
```
my_flask_app/
├── app/
│   ├── __init__.py
│   ├── routes.py
│   ├── models.py
│   └── templates/
│       └── index.html
├── config.py
└── run.py
```

In [ ]:
# Flask 애플리케이션 예제
from flask import Flask, render_template, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy

class FlaskBlogApp:
    def __init__(self):
        self.app = Flask(__name__)
        self.app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///blog.db'
        self.db = SQLAlchemy(self.app)
        self.setup_routes()
        
    def setup_routes(self):
        """라우트 설정"""
        @self.app.route('/')
        def home():
            posts = Post.query.all()
            return render_template('home.html', posts=posts)
        
        @self.app.route('/post/new', methods=['GET', 'POST'])
        def new_post():
            if request.method == 'POST':
                post = Post(
                    title=request.form['title'],
                    content=request.form['content']
                )
                self.db.session.add(post)
                self.db.session.commit()
                return redirect(url_for('home'))
            return render_template('new_post.html')
    
    def run(self):
        """애플리케이션 실행"""
        self.app.run(debug=True)

## 13.3 Django 웹 프로젝트 구축

### ✅ 13.3.1 Django MTV 패턴
- Model: 데이터베이스 구조
- Template: 화면 표시
- View: 비즈니스 로직

In [ ]:
# Django 모델 예제
from django.db import models
from django.contrib.auth.models import User

class BlogPost(models.Model):
    """블로그 포스트 모델"""
    title = models.CharField(max_length=200)
    content = models.TextField()
    author = models.ForeignKey(User, on_delete=models.CASCADE)
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    def __str__(self):
        return self.title

# Django 뷰 예제
from django.views.generic import ListView, CreateView
from django.urls import reverse_lazy

class BlogListView(ListView):
    """블로그 목록 뷰"""
    model = BlogPost
    template_name = 'blog/post_list.html'
    context_object_name = 'posts'
    ordering = ['-created_at']

class BlogCreateView(CreateView):
    """블로그 작성 뷰"""
    model = BlogPost
    template_name = 'blog/post_form.html'
    fields = ['title', 'content']
    success_url = reverse_lazy('post-list')

## 13.4 REST API 개발

### ✅ 13.4.1 API 엔드포인트 설계
- GET /api/posts/: 포스트 목록 조회
- POST /api/posts/: 새 포스트 생성
- GET /api/posts/{id}/: 특정 포스트 조회
- PUT /api/posts/{id}/: 포스트 수정
- DELETE /api/posts/{id}/: 포스트 삭제

In [ ]:
# Django REST Framework 뷰셋
from rest_framework import viewsets
from rest_framework import permissions
from .serializers import BlogPostSerializer

class BlogPostViewSet(viewsets.ModelViewSet):
    """블로그 포스트 API 뷰셋"""
    queryset = BlogPost.objects.all()
    serializer_class = BlogPostSerializer
    permission_classes = [permissions.IsAuthenticatedOrReadOnly]
    
    def perform_create(self, serializer):
        serializer.save(author=self.request.user)

## 🎯 실습 프로젝트

### [실습 1] Flask 블로그 애플리케이션 구현

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime

def create_blog_app():
    """블로그 애플리케이션 생성"""
    app = Flask(__name__)
    app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///blog.db'
    db = SQLAlchemy(app)
    
    class Post(db.Model):
        id = db.Column(db.Integer, primary_key=True)
        title = db.Column(db.String(100), nullable=False)
        content = db.Column(db.Text, nullable=False)
        created_at = db.Column(db.DateTime, default=datetime.utcnow)
    
    @app.route('/')
    def home():
        posts = Post.query.order_by(Post.created_at.desc()).all()
        return render_template('home.html', posts=posts)
    
    return app

### [실습 2] Django REST API 구현

In [ ]:
from rest_framework import generics
from rest_framework import permissions

class PostListCreateAPI(generics.ListCreateAPIView):
    """포스트 목록 조회 및 생성 API"""
    queryset = Post.objects.all()
    serializer_class = PostSerializer
    permission_classes = [permissions.IsAuthenticatedOrReadOnly]
    
    def perform_create(self, serializer):
        serializer.save(author=self.request.user)

class PostDetailAPI(generics.RetrieveUpdateDestroyAPIView):
    """포스트 상세 조회, 수정, 삭제 API"""
    queryset = Post.objects.all()
    serializer_class = PostSerializer
    permission_classes = [permissions.IsAuthenticatedOrReadOnly]